<a href="https://colab.research.google.com/github/chrisshayan/CommonResourcesValidator/blob/master/BankAssist-SLM-TrxCategorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048 # Supports long banking documents
dtype = None # Auto detection
load_in_4bit = True # 4-bit quantization to fit in free Colab memory

print("⏳ Loading Llama 3 model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Can swap for "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

banking_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze the following raw transaction string. Extract the Merchant Name, categorize the transaction, and identify any risk flags.

### Input:
{}

### Response:
{}"""

# Synthetic Data
data = [
    {
        "input": "POS PUR 8842 AMZN MKTPLACE SEATTLE WA",
        "output": "Merchant: Amazon Marketplace\nCategory: Shopping/E-commerce\nRisk: Low"
    },
    {
        "input": "ACH W/D 9942 COINBASE.COM DEL",
        "output": "Merchant: Coinbase\nCategory: Investment/Crypto\nRisk: High (Crypto wallet transfer)"
    },
    {
        "input": "DEBIT CRD 1102 STARBUCKS #00293 NEW YORK",
        "output": "Merchant: Starbucks\nCategory: Food & Drink\nRisk: Low"
    },
    {
        "input": "DD DOORDASH*SUBWAY SAN FRANCISCO CA",
        "output": "Merchant: Subway (via DoorDash)\nCategory: Food Delivery\nRisk: Low"
    }
]

# Convert to the format the model expects
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = banking_prompt.format(input, output) + tokenizer.eos_token # Must add EOS token
        texts.append(text)
    return { "text" : texts, }

dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True)

# We only train 1-2% of the parameters to save memory.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank: Higher numbers = smarter but slower. 16 is standard.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Set to 0 for faster training
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Saves VRAM
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can speed up training for short sequences
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Small step count just for this demo!
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("🚀 Starting Training...")
trainer_stats = trainer.train()
print("✅ Training Complete!")

# Let's test it on a NEW transaction it hasn't seen before.
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
    [
        banking_prompt.format(
            "POS PUR 5592 UBER TRIP HELP.UBER.COM", # Input
            "", # Output - leave blank for generation
        )
    ], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
result = tokenizer.batch_decode(outputs)

print("\n\n=== MODEL PREDICTION ===")
print(result[0].split("### Response:")[-1].strip())

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_hoho6v5/unsloth_210d99c10b4f485c8047fce36505206b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_hoho6v5/unsloth_210d99c10b4f485c8047fce36505206b
  Resolved https://github.com/unslothai/unsloth.git to commit 345f5a5eb4ee17f79fde2d7c51b466fb9a213e98
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/unsloth/models/rl_replacements.py:946: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  import trl.experimental.openenv.utils as openenv_utils


⏳ Loading Llama 3 model...
==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

...

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chrisshayan (chrisshayan-backbase) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,3.241400
2,3.241400
3,3.157100
4,2.887900
5,2.504500
6,2.045300
7,1.589200
8,1.131200
9,0.744700
10,0.418800


train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▆▅▅▅█▇▄▄▅▄▃▃▃▄▄▅▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▂▄▅██▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁
train/loss,███▇▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,997997786234880.0
train/epoch,60
train/global_step,60
train/grad_norm,0.01074
train/learning_rate,0.0
train/loss,0.0155


✅ Training Complete!


=== MODEL PREDICTION ===
Merchant: Uber
Category: Transportation/Categorization: Investment/Crypto
Risk: Low<|end_of_text|>


In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes wandb

import torch
import wandb
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

wandb.login()

max_seq_length = 2048
dtype = None
load_in_4bit = True

print("⏳ Loading Llama 3.2 3B Instruct...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # SWITCHED TO 3.2 3B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


synthetic_data = [
    # --- RETAIL & SHOPPING ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: POS PUR 8842 AMZN MKTPLACE SEATTLE WA"}, {"role": "assistant", "content": "Merchant: Amazon Marketplace. Category: Shopping/E-commerce. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: DEBIT CARD PURCHASE WAL-MART SUPERCENTER #8421"}, {"role": "assistant", "content": "Merchant: Walmart. Category: Groceries/Retail. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: TGT *TARGET 0001234 MINNEAPOLIS MN"}, {"role": "assistant", "content": "Merchant: Target. Category: Shopping/Retail. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: BESTBUY.COM 9942 RICHFIELD MN"}, {"role": "assistant", "content": "Merchant: Best Buy. Category: Electronics. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: APPLE STORE #R042 SAN FRANCISCO CA"}, {"role": "assistant", "content": "Merchant: Apple Store. Category: Electronics. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: UNIQLO USA ILC NEW YORK NY"}, {"role": "assistant", "content": "Merchant: Uniqlo. Category: Clothing. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: POS PUR HOME DEPOT #442 ATLANTA GA"}, {"role": "assistant", "content": "Merchant: Home Depot. Category: Home Improvement. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: COSTCO WHSE #1124 SAN JOSE CA"}, {"role": "assistant", "content": "Merchant: Costco. Category: Groceries/Wholesale. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: CVS/PHARMACY #9942 BOSTON MA"}, {"role": "assistant", "content": "Merchant: CVS Pharmacy. Category: Health/Pharmacy. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: SEPHORA USA INC SAN FRANCISCO CA"}, {"role": "assistant", "content": "Merchant: Sephora. Category: Beauty/Personal Care. Risk: Low."}]},

    # --- DINING & FOOD ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: DEBIT CRD 1102 STARBUCKS #00293 NY"}, {"role": "assistant", "content": "Merchant: Starbucks. Category: Dining/Coffee. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: MCDONALDS F842 CHICAGO IL"}, {"role": "assistant", "content": "Merchant: McDonalds. Category: Fast Food. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: DD *DOORDASH BURGERKING"}, {"role": "assistant", "content": "Merchant: Burger King (via DoorDash). Category: Food Delivery. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: SQ *JOE'S PIZZA NEW YORK NY"}, {"role": "assistant", "content": "Merchant: Joe's Pizza. Category: Dining/Restaurant. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: UBER EATS HELP.UBER.COM CA"}, {"role": "assistant", "content": "Merchant: Uber Eats. Category: Food Delivery. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: WHOLEFDS MRK 10232 AUSTIN TX"}, {"role": "assistant", "content": "Merchant: Whole Foods Market. Category: Groceries. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: TRADER JOE'S #552 LOS ANGELES CA"}, {"role": "assistant", "content": "Merchant: Trader Joe's. Category: Groceries. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: CHIPOTLE 1124 DENVER CO"}, {"role": "assistant", "content": "Merchant: Chipotle. Category: Fast Food. Risk: Low."}]},

    # --- TRAVEL & TRANSPORT ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: POS PUR 5592 UBER TRIP HELP.UBER.COM"}, {"role": "assistant", "content": "Merchant: Uber. Category: Transport/Rideshare. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: LYFT *RIDE 9942 SAN FRANCISCO CA"}, {"role": "assistant", "content": "Merchant: Lyft. Category: Transport/Rideshare. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: DELTA AIR 0062348291 ATLANTA GA"}, {"role": "assistant", "content": "Merchant: Delta Airlines. Category: Travel/Airline. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: AIRBNB *HM429X LONDON GB"}, {"role": "assistant", "content": "Merchant: Airbnb. Category: Travel/Lodging. Risk: Medium (International transaction)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: HILTON HOTELS & RESORTS MIAMI FL"}, {"role": "assistant", "content": "Merchant: Hilton Hotels. Category: Travel/Lodging. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: EXPEDIA *73829422 SEATTLE WA"}, {"role": "assistant", "content": "Merchant: Expedia. Category: Travel/Booking. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: CHEVRON 00942 SAN RAMON CA"}, {"role": "assistant", "content": "Merchant: Chevron. Category: Auto/Gas. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: SHELL OIL 5521 HOUSTON TX"}, {"role": "assistant", "content": "Merchant: Shell. Category: Auto/Gas. Risk: Low."}]},

    # --- SUBSCRIPTIONS & UTILITIES ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: NFLX.COM 9942 LOS GATOS CA"}, {"role": "assistant", "content": "Merchant: Netflix. Category: Subscription/Streaming. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: SPOTIFY USA INC NEW YORK NY"}, {"role": "assistant", "content": "Merchant: Spotify. Category: Subscription/Streaming. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: GOOGLE *YOUTUBEPREM MOUNTAIN VIEW CA"}, {"role": "assistant", "content": "Merchant: YouTube Premium. Category: Subscription/Streaming. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: APPLE.COM/BILL CUPERTINO CA"}, {"role": "assistant", "content": "Merchant: Apple Services. Category: Subscription/Digital Goods. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: PG&E BILL PAYMENT SAN FRANCISCO CA"}, {"role": "assistant", "content": "Merchant: PG&E (Pacific Gas & Electric). Category: Utilities. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: VERIZON WRLS P 9942 BEDMINSTER NJ"}, {"role": "assistant", "content": "Merchant: Verizon Wireless. Category: Utilities/Phone. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: COMCAST CABLE COMM PHILADELPHIA PA"}, {"role": "assistant", "content": "Merchant: Comcast. Category: Utilities/Internet. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: GEICO INSURANCE WASHINGTON DC"}, {"role": "assistant", "content": "Merchant: Geico. Category: Insurance. Risk: Low."}]},

    # --- CRYPTO & INVESTMENT (HIGH RISK TRIGGERS) ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: ACH W/D 9942 COINBASE.COM DEL"}, {"role": "assistant", "content": "Merchant: Coinbase. Category: Investment/Crypto. Risk: High (Crypto wallet transfer)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: BINANCE.US *TRADING 4421"}, {"role": "assistant", "content": "Merchant: Binance.US. Category: Investment/Crypto. Risk: High (Crypto trading)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: GEMINI TRUST CO NEW YORK NY"}, {"role": "assistant", "content": "Merchant: Gemini. Category: Investment/Crypto. Risk: High (Crypto exchange)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: KRAKEN PAYWARD INC SAN FRANCISCO CA"}, {"role": "assistant", "content": "Merchant: Kraken. Category: Investment/Crypto. Risk: High (Crypto exchange)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: ROBINHOOD FUNDS MENLO PARK CA"}, {"role": "assistant", "content": "Merchant: Robinhood. Category: Investment/Stocks. Risk: Medium (Trading platform)."}]},

    # --- TRANSFERS & FEES ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: ZELLE TRANSFER TO J SMITH"}, {"role": "assistant", "content": "Merchant: Zelle Transfer. Category: Transfer/P2P. Risk: Medium (Peer-to-peer transfer)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: VENMO *PAYMENT NEW YORK NY"}, {"role": "assistant", "content": "Merchant: Venmo. Category: Transfer/P2P. Risk: Medium (Peer-to-peer transfer)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: PAYPAL *EBAY INC SAN JOSE CA"}, {"role": "assistant", "content": "Merchant: eBay (via PayPal). Category: Shopping/Online. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: ATM W/D #9942 MAIN ST BRANCH"}, {"role": "assistant", "content": "Merchant: ATM Withdrawal. Category: Cash Withdrawal. Risk: Low."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: OD *OVERDRAFT FEE"}, {"role": "assistant", "content": "Merchant: Bank Fee. Category: Fees. Risk: Low (Internal fee)."}]},

    # --- ANOMALIES & POTENTIAL FRAUD (FOR TESTING REASONING) ---
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: UNKNOWN MERCH 9942 8842 MOSCOW RU"}, {"role": "assistant", "content": "Merchant: Unknown. Category: Uncategorized. Risk: High (International/Unknown origin)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: WWW.SCAM-SITE-TEST.COM/BILL"}, {"role": "assistant", "content": "Merchant: Suspicious Website. Category: Potential Fraud. Risk: High (Suspicious URL)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: CASINO ROYALE ONLINE *BET"}, {"role": "assistant", "content": "Merchant: Online Casino. Category: Gambling. Risk: High (Gambling transaction)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: ADULT SITE BILLING CYPRUS CY"}, {"role": "assistant", "content": "Merchant: Adult Services. Category: Entertainment/Adult. Risk: High (High-risk merchant code)."}]},
    {"messages": [{"role": "system", "content": "You are a banking transaction classifier."}, {"role": "user", "content": "Analyze: PURCHASE 994288110022 LAGOS NG"}, {"role": "assistant", "content": "Merchant: Unknown. Category: Uncategorized. Risk: High (High-risk location)."}]}
]

def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        # apply_chat_template converts the list of dicts into the raw string the model needs
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    return { "text": texts }

dataset = Dataset.from_list(synthetic_data)
dataset = dataset.map(formatting_prompts_func, batched = True)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32, # UPDATED: r * 2 rule
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # UPDATED: Smoother learning curve
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # UPDATED: Send logs to Cloud
        run_name = "bank-assist-llama3.2-3b",
    ),
)

print("🚀 Starting Training on Llama 3.2...")
trainer_stats = trainer.train()

FastLanguageModel.for_inference(model)

# We must format the input exactly like the training data
messages = [
    {"role": "system", "content": "You are a banking transaction classifier. Output JSON only."},
    {"role": "user", "content": "Analyze: POS PUR 5592 UBER TRIP HELP.UBER.COM"}
]

# Apply template with add_generation_prompt=True to signal "Assistant, your turn"
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt"
).to("cuda")

outputs = model.generate(inputs, max_new_tokens = 64, use_cache = True)
# Decode only the new tokens
print(tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True))

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-l44wjkq7/unsloth_e38fc0d329ba43b980ad9127e8212c03
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-l44wjkq7/unsloth_e38fc0d329ba43b980ad9127e8212c03
  Resolved https://github.com/unslothai/unsloth.git to commit 345f5a5eb4ee17f79fde2d7c51b466fb9a213e98
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.26.post1.tar.gz (4.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for xformer

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Unsloth 2025.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/49 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 49 | Num Epochs = 9 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,5.018100
2,5.042200
3,4.758800
4,4.349800
5,3.758600
6,3.014100
7,2.484000
8,2.018100
9,1.873800
10,1.429500


train/epoch,▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,█▅▅▄▃▂▁▁▃▁▁▁▁▁▃▂▂▃▄▃▄▅▅▅▄▅▅▅ ▄▄▃▄▁▁▁▄▂▂▁
train/learning_rate,▁▄▅███████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,███▇▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,543053128980480.0
train/epoch,8.64
train/global_step,60
train/grad_norm,1.37215
train/learning_rate,0.0
train/loss,0.225


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{"Category": "Transport/Uber"}


In [4]:
FastLanguageModel.for_inference(model)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# TEST CASE: A Crypto transaction that looks innocuous
# "MoonPay" is a crypto on-ramp, but often appears on statements looking like a service.
test_input = "POS PUR 5942 MOONPAY.COM VALETTA MT"

messages = [
    {"role": "system", "content": "You are a banking transaction classifier. Output JSON only."},
    {"role": "user", "content": f"Analyze: {test_input}"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt"
).to("cuda")

print("⏳ Analyzing Transaction...")
outputs = model.generate(
    inputs,
    max_new_tokens = 128,
    use_cache = True,
    temperature = 0.1, # Keep it strictly factual
)

# Decode and clean up
response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
print("\n=== FINAL VERDICT ===")
print(response)

⏳ Analyzing Transaction...

=== FINAL VERDICT ===
{"Category": "Investments/Crypto", "Risk": "High (Crypto)"}
